In [1]:
# from google.colab import drive
# drive.mount('/content/drive')
# !unzip "/content/drive/MyDrive/open.zip"

In [2]:
import os
import sys
import numpy as np
from PIL import ImageFile
from datetime import datetime

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.data.experimental import cardinality
# from tensorflow.data.Dataset import from_tensor_slices
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.image import random_brightness, random_contrast, random_crop, random_flip_left_right, random_flip_up_down, random_hue, random_jpeg_quality, random_saturation

sys.path.append('/content/drive/MyDrive/Colab Notebooks/')
# sys.path.append(os.path.join(os.getcwd(), os.path.pardir, 'functions'))

from get_tensor_through_imgs_fn_v2 import get_tensor_through_imgs_fn
from get_train_image_paths_and_labels_fn import get_train_image_paths_and_labels_fn

In [3]:
# ======== 학습 데이터 경로 리스트 + 라벨 데이터 정규화, 원핫 인코딩 리스트 ========

train_image_paths, labels = get_train_image_paths_and_labels_fn()

# ===============================================================================

# ========= 학습 이미지 로드, 정규화, 사이즈 조정 =========

# 잘린 이미지에 대한 로드 여부 설정
ImageFile.LOAD_TRUNCATED_IMAGES = True

RESIZED_WIDTH = 224
RESIZED_HEIGHT = 224

train_image_tensor = get_tensor_through_imgs_fn(train_image_paths, RESIZED_WIDTH, RESIZED_HEIGHT)
print('train_image_tensor.shape :', train_image_tensor.shape)

# =======================================================


encoding_labels.shape : (3457, 19)


100%|██████████| 3457/3457 [01:15<00:00, 45.58it/s]


train_image_tensor.shape : (3457, 224, 224, 3)


In [4]:
del train_image_paths

In [5]:
# ============ EfficientNetB0() + 이미지 증강 사용해보기 ============

TRAIN_FOLDER_PATH = '/content/train/'
TEST_FOLDER_PATH = '/content/test/'
# TRAIN_FOLDER_PATH = '../../open/train/'
# TEST_FOLDER_PATH = '../../open/test/'

EPOCHS = 10

train_ds = tf.data.Dataset.from_tensor_slices((train_image_tensor, labels))

train_ds_size = cardinality(train_ds).numpy()
print("Training data size:", train_ds_size)

train_ds = (train_ds
            .shuffle(buffer_size=train_ds_size)
            .batch(batch_size=32, drop_remainder=True))

model = Sequential()
model.add(EfficientNetB0(input_shape=(RESIZED_WIDTH,
          RESIZED_HEIGHT, 3), include_top=False, weights=None))
model.add(Flatten())
model.add(Dense(19, activation='softmax'))

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy', metrics=['acc'])

model.summary()

model.fit(train_ds, epochs=EPOCHS)

# 연월일_시간
now = datetime.today().strftime('%Y%m%d_%H%M%S')
model.save(f'model_{now}.h5')

Training data size: 3457
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional)  (None, 7, 7, 1280)       4049571   
                                                                 
 flatten (Flatten)           (None, 62720)             0         
                                                                 
 dense (Dense)               (None, 19)                1191699   
                                                                 
Total params: 5,241,270
Trainable params: 5,199,247
Non-trainable params: 42,023
_________________________________________________________________
Epoch 1/10
108/108 [==============================] - 78s 252ms/step - loss: 2.3072 - acc: 0.3637
Epoch 2/10
108/108 [==============================] - 28s 255ms/step - loss: 2.0182 - acc: 0.3981
Epoch 3/10
108/108 [==============================] - 28s 258ms/step - loss: 1.8035 

In [6]:
del train_ds

In [7]:
# random_brightness, random_contrast, random_crop, random_flip_left_right, random_flip_up_down, random_hue, random_jpeg_quality, random_saturation

def train_augmentation_images_fn(augmentation_image_list):
  augmentation_ds = tf.data.Dataset.from_tensor_slices((np.array(augmentation_image_list), labels))
  augmentation_ds = (augmentation_ds
            .shuffle(buffer_size=train_ds_size)
            .batch(batch_size=32, drop_remainder=True))
  model.fit(augmentation_ds, epochs=EPOCHS)

In [8]:
augmentation_image_list = []

for index in range(len(train_image_tensor)):
  augmentation_image = random_brightness(train_image_tensor[index], 0.2)
  augmentation_image_list.append(augmentation_image)

train_augmentation_images_fn(augmentation_image_list)

Epoch 1/10
108/108 [==============================] - 28s 262ms/step - loss: 2.6840 - acc: 0.3160
Epoch 2/10
108/108 [==============================] - 28s 262ms/step - loss: 2.4902 - acc: 0.3579
Epoch 3/10
108/108 [==============================] - 28s 260ms/step - loss: 2.2323 - acc: 0.3681
Epoch 4/10
108/108 [==============================] - 28s 260ms/step - loss: 2.1404 - acc: 0.3822
Epoch 5/10
108/108 [==============================] - 28s 261ms/step - loss: 2.0847 - acc: 0.3857
Epoch 6/10
108/108 [==============================] - 28s 261ms/step - loss: 2.0403 - acc: 0.3863
Epoch 7/10
108/108 [==============================] - 28s 261ms/step - loss: 1.9563 - acc: 0.3915
Epoch 8/10
108/108 [==============================] - 28s 262ms/step - loss: 1.9084 - acc: 0.4034
Epoch 9/10
108/108 [==============================] - 28s 261ms/step - loss: 1.8382 - acc: 0.4196
Epoch 10/10
108/108 [==============================] - 28s 261ms/step - loss: 1.8153 - acc: 0.4201


In [10]:
augmentation_image_list = []

for index in range(len(train_image_tensor)):
  augmentation_image = random_contrast(train_image_tensor[index], 0.2, 0.5)
  augmentation_image_list.append(augmentation_image)

train_augmentation_images_fn(augmentation_image_list)

Epoch 1/10
108/108 [==============================] - 28s 261ms/step - loss: 1.8987 - acc: 0.3947
Epoch 2/10
108/108 [==============================] - 28s 262ms/step - loss: 1.6747 - acc: 0.4502
Epoch 3/10
108/108 [==============================] - 28s 259ms/step - loss: 1.6043 - acc: 0.4731
Epoch 4/10
108/108 [==============================] - 28s 260ms/step - loss: 1.4392 - acc: 0.5078
Epoch 5/10
108/108 [==============================] - 28s 261ms/step - loss: 1.2783 - acc: 0.5677
Epoch 6/10
108/108 [==============================] - 28s 260ms/step - loss: 1.0922 - acc: 0.6296
Epoch 7/10
108/108 [==============================] - 28s 260ms/step - loss: 0.9166 - acc: 0.6962
Epoch 8/10
108/108 [==============================] - 28s 260ms/step - loss: 0.7492 - acc: 0.7500
Epoch 9/10
108/108 [==============================] - 28s 261ms/step - loss: 0.7033 - acc: 0.7595
Epoch 10/10
108/108 [==============================] - 28s 260ms/step - loss: 0.6135 - acc: 0.7989


In [11]:
augmentation_image_list = []

for index in range(len(train_image_tensor)):
  augmentation_image = random_flip_left_right(train_image_tensor[index])
  augmentation_image_list.append(augmentation_image)

train_augmentation_images_fn(augmentation_image_list)

Epoch 1/10
108/108 [==============================] - 28s 260ms/step - loss: 1.6179 - acc: 0.5255
Epoch 2/10
108/108 [==============================] - 28s 262ms/step - loss: 1.0394 - acc: 0.6623
Epoch 3/10
108/108 [==============================] - 28s 259ms/step - loss: 0.7238 - acc: 0.7760
Epoch 4/10
108/108 [==============================] - 28s 260ms/step - loss: 0.5901 - acc: 0.8116
Epoch 5/10
108/108 [==============================] - 28s 261ms/step - loss: 0.4838 - acc: 0.8423
Epoch 6/10
108/108 [==============================] - 28s 260ms/step - loss: 0.4153 - acc: 0.8611
Epoch 7/10
108/108 [==============================] - 28s 260ms/step - loss: 0.3399 - acc: 0.8909
Epoch 8/10
108/108 [==============================] - 28s 261ms/step - loss: 0.3074 - acc: 0.8993
Epoch 9/10
108/108 [==============================] - 28s 261ms/step - loss: 0.3110 - acc: 0.9005
Epoch 10/10
108/108 [==============================] - 28s 260ms/step - loss: 0.3116 - acc: 0.9010


In [12]:
augmentation_image_list = []

for index in range(len(train_image_tensor)):
  augmentation_image = random_flip_up_down(train_image_tensor[index])
  augmentation_image_list.append(augmentation_image)

train_augmentation_images_fn(augmentation_image_list)

Epoch 1/10
108/108 [==============================] - 28s 261ms/step - loss: 1.7277 - acc: 0.5437
Epoch 2/10
108/108 [==============================] - 28s 262ms/step - loss: 0.9501 - acc: 0.7028
Epoch 3/10
108/108 [==============================] - 28s 259ms/step - loss: 0.6868 - acc: 0.7905
Epoch 4/10
108/108 [==============================] - 28s 260ms/step - loss: 0.5295 - acc: 0.8310
Epoch 5/10
108/108 [==============================] - 28s 261ms/step - loss: 0.4379 - acc: 0.8588
Epoch 6/10
108/108 [==============================] - 28s 260ms/step - loss: 0.3940 - acc: 0.8718
Epoch 7/10
108/108 [==============================] - 28s 260ms/step - loss: 0.3501 - acc: 0.8883
Epoch 8/10
108/108 [==============================] - 28s 260ms/step - loss: 0.3265 - acc: 0.8924
Epoch 9/10
108/108 [==============================] - 28s 261ms/step - loss: 0.3092 - acc: 0.9060
Epoch 10/10
108/108 [==============================] - 28s 262ms/step - loss: 0.2653 - acc: 0.9141


In [13]:
augmentation_image_list = []

for index in range(len(train_image_tensor)):
  augmentation_image = random_hue(train_image_tensor[index], 0.2)
  augmentation_image_list.append(augmentation_image)

train_augmentation_images_fn(augmentation_image_list)

Epoch 1/10
108/108 [==============================] - 28s 260ms/step - loss: 0.8950 - acc: 0.7277
Epoch 2/10
108/108 [==============================] - 28s 261ms/step - loss: 0.5060 - acc: 0.8235
Epoch 3/10
108/108 [==============================] - 28s 260ms/step - loss: 0.3414 - acc: 0.8845
Epoch 4/10
108/108 [==============================] - 28s 263ms/step - loss: 0.3206 - acc: 0.9016
Epoch 5/10
108/108 [==============================] - 28s 260ms/step - loss: 0.2485 - acc: 0.9181
Epoch 6/10
108/108 [==============================] - 28s 260ms/step - loss: 0.2259 - acc: 0.9303
Epoch 7/10
108/108 [==============================] - 28s 261ms/step - loss: 0.2215 - acc: 0.9314
Epoch 8/10
108/108 [==============================] - 28s 260ms/step - loss: 0.2005 - acc: 0.9352
Epoch 9/10
108/108 [==============================] - 28s 261ms/step - loss: 0.1905 - acc: 0.9401
Epoch 10/10
108/108 [==============================] - 28s 260ms/step - loss: 0.1729 - acc: 0.9447


In [14]:
augmentation_image_list = []

for index in range(len(train_image_tensor)):
  augmentation_image = random_jpeg_quality(train_image_tensor[index], 75, 95)
  augmentation_image_list.append(augmentation_image)

train_augmentation_images_fn(augmentation_image_list)

Epoch 1/10
108/108 [==============================] - 28s 260ms/step - loss: 2.6763 - acc: 0.3203
Epoch 2/10
108/108 [==============================] - 28s 262ms/step - loss: 1.8921 - acc: 0.4132
Epoch 3/10
108/108 [==============================] - 28s 259ms/step - loss: 1.7069 - acc: 0.4479
Epoch 4/10
108/108 [==============================] - 28s 260ms/step - loss: 1.5146 - acc: 0.5139
Epoch 5/10
108/108 [==============================] - 28s 261ms/step - loss: 1.3802 - acc: 0.5602
Epoch 6/10
108/108 [==============================] - 28s 260ms/step - loss: 1.1466 - acc: 0.6314
Epoch 7/10
108/108 [==============================] - 28s 260ms/step - loss: 0.9923 - acc: 0.6811
Epoch 8/10
108/108 [==============================] - 28s 260ms/step - loss: 0.8881 - acc: 0.7098
Epoch 9/10
108/108 [==============================] - 28s 260ms/step - loss: 0.8161 - acc: 0.7468
Epoch 10/10
108/108 [==============================] - 28s 260ms/step - loss: 0.7521 - acc: 0.7616


In [15]:
augmentation_image_list = []

for index in range(len(train_image_tensor)):
  augmentation_image = random_saturation(train_image_tensor[index], 5, 10)
  augmentation_image_list.append(augmentation_image)

train_augmentation_images_fn(augmentation_image_list)

Epoch 1/10
108/108 [==============================] - 28s 261ms/step - loss: 1.3728 - acc: 0.5596
Epoch 2/10
108/108 [==============================] - 28s 262ms/step - loss: 0.6781 - acc: 0.7659
Epoch 3/10
108/108 [==============================] - 28s 260ms/step - loss: 0.4462 - acc: 0.8461
Epoch 4/10
108/108 [==============================] - 28s 261ms/step - loss: 0.3670 - acc: 0.8776
Epoch 5/10
108/108 [==============================] - 28s 263ms/step - loss: 0.2913 - acc: 0.9034
Epoch 6/10
108/108 [==============================] - 28s 260ms/step - loss: 0.2368 - acc: 0.9222
Epoch 7/10
108/108 [==============================] - 28s 263ms/step - loss: 0.2082 - acc: 0.9308
Epoch 8/10
108/108 [==============================] - 28s 260ms/step - loss: 0.2000 - acc: 0.9398
Epoch 9/10
108/108 [==============================] - 28s 260ms/step - loss: 0.1688 - acc: 0.9485
Epoch 10/10
108/108 [==============================] - 28s 261ms/step - loss: 0.1717 - acc: 0.9430


In [17]:
# 연월일_시간
now = datetime.today().strftime('%Y%m%d_%H%M%S')
model.save(f'model_{now}.h5')